In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
import numpy as np
import pandas as pd

In [18]:
# Load the dataset
file_path = 'new.csv'
df = pd.read_csv(file_path)

# Separate features and target
X = df.iloc[:, 1:]  # Exclude the first column which is the target
y = df.iloc[:, 0]   # The first column is the target

# Split the data into training and validation sets
train_size = int(len(df) * 0.8)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

# Further split the training data for time series cross-validation
train_subset_size = int(len(X_train) * 0.5)
# Initialize an empty list to store evaluation metrics for each iteration
results = []
model = LinearRegression()

# Loop through the training data, expanding the training set and using the next 40 rows as the validation set
for i in range(train_subset_size, len(X_train) - 80, 80):
    # Define the current training and validation subsets
    X_train_current = X_train.iloc[:i]
    y_train_current = y_train.iloc[:i]
    X_val_current = X_train.iloc[i:i+80]
    y_val_current = y_train.iloc[i:i+80]
    
    # Fit the model on the current training subset
    model.fit(X_train_current, y_train_current)
    
    # Make predictions on the current validation subset
    y_pred_val_current = model.predict(X_val_current)
    # y_pred_val_current to series
    y_pred_val_current = pd.Series(y_pred_val_current)
    # print(y_val_current)
    # put y_pred_val_current and y_val_current in a dataframe
    y_pred_da = pd.DataFrame(y_pred_val_current, columns=['y_pred'])
    y_val_da = pd.DataFrame(y_val_current)
    # reset index
    y_pred_da.reset_index(drop=True, inplace=True)
    y_val_da.reset_index(drop=True, inplace=True)
    # print(y_pred_da)
    # print(y_val_da)
    # combine
    y_da = pd.concat([y_pred_da, y_val_da], axis=1)
    
    # new column flag, if both column > 0 or both column < 0, then 1, else 0
    y_da['flag'] = np.where((y_da['y_pred'] > 0) & (y_da['label'] > 0) | (y_da['y_pred'] < 0) & (y_da['label'] < 0), 1, 0)
    print(y_da)
    print(y_da['flag'].sum())
    # sharpe_ratio_current = np.mean(np.abs(y_pred_da)) / np.std(y_pred_da)
    # print(sharpe_ratio_current)
    # accuracy_current = accuracy_score((y_val_da > 0).astype(int), (y_pred_da > 0).astype(int))
    # print(accuracy_current)
    # r2_current = r2_score(y_val_da, y_pred_da)
    # print(r2_current)
    break

    # Calculate R2 statistic, Sharpe ratio, and accuracy on the current validation subset
    r2_current = r2_score(y_val_current, y_pred_val_current)
    sharpe_ratio_current = np.mean(np.abs(y_pred_val_current)) / np.std(y_pred_val_current)
    accuracy_current = accuracy_score((y_val_current > 0).astype(int), (y_pred_val_current > 0).astype(int))
    
    # Append the metrics to the results list
    results.append({
        'Train Size': i,
        'R2 Statistic': r2_current,
        'Sharpe Ratio': sharpe_ratio_current,
        'Accuracy': accuracy_current
    })

# Convert the results list to a DataFrame for easier viewing
results_df = pd.DataFrame(results)

# Display the results
results_df


      y_pred     label  flag
0  -0.000689  0.000316     0
1  -0.000661 -0.011055     1
2  -0.000898 -0.013095     1
3  -0.001507 -0.003236     1
4  -0.000974 -0.003896     1
..       ...       ...   ...
75 -0.001313 -0.009444     1
76 -0.001376  0.001021     0
77 -0.001211 -0.008503     1
78 -0.001524 -0.001715     1
79 -0.001639 -0.019588     1

[80 rows x 3 columns]
42


""
